In [ ]:

import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.ensemble import IsolationForest

df = pd.read_csv('./REF_Buchungen_2018_Datenset_A.csv',sep=';',header=None, encoding='ISO-8859-1', low_memory=False) 

df.rename({0: 'KtoNr', 4:'Datum', 8:'Umsatz_S', 9:'Umsatz_H', 19:'Anwender'}, axis=1, inplace=True) 
#df.rename({0: 'KtoNr', 6:'Umsatz_S', 7:'Umsatz_H',18:'Anwender'}, axis=1, inplace=True)
 
# Filtern der Spalten
df= df[['KtoNr','Umsatz_S', 'Umsatz_H', 'Anwender']]

#Filtern der Kontonummern für den Kontenbereich Umsatzerlöse
df = df[((df['KtoNr'] >= 8000) & (df['KtoNr'] <= 8589)) | ((df['KtoNr'] >= 8900) & (df['KtoNr'] <= 8919)) | ((df['KtoNr'] >= 8940) & (df['KtoNr'] <= 8959)) |
   ((df['KtoNr'] >= 8700) & (df['KtoNr'] <= 8799))].copy()

#Handle missing values
df['KtoNr'].dropna(inplace=True)
df['Umsatz_S'].fillna(0, inplace=True)
df['Umsatz_H'].fillna(0, inplace=True)
df['Anwender'].fillna('Unbekannt', inplace=True)

#Datentyptransformationen
df['Umsatz_H'] = df['Umsatz_H'].str.replace(',','.')
df['Umsatz_H'] = pd.to_numeric(df['Umsatz_H'])

df['Umsatz_S'] = df['Umsatz_S'].str.replace(',','.')
df['Umsatz_S'] = pd.to_numeric(df['Umsatz_S'])

df['Anwender'] = df.Anwender.astype('category')
df['Anwender'] = df['Anwender'].cat.codes

#Berechnung des Saldos
df['Saldo'] = df.apply(lambda row: row.Umsatz_H - row.Umsatz_S, axis = 1) 
df = df.drop(columns=['Umsatz_S','Umsatz_H', 'KtoNr'])


df_Umsaetze = df
df_Umsaetze.info()
        



In [ ]:
#df_Umsaetze['Datum'] = pd.to_datetime(df_Umsaetze['Datum'],format='%d.%m.%Y')
#df_Umsaetze.sort_values(by=['Datum'], inplace=True)

In [ ]:
#Sortieren der Zahlen nach Datum
#df_Umsaetze = df_Umsaetze.set_index(pd.DatetimeIndex(df_Umsaetze['Datum']))
#df_Umsaetze = df_Umsaetze['Saldo'].resample("D").sum()

sns.set(style="whitegrid")
sns.set(style="whitegrid",rc={'figure.figsize':(20,9)})
ax = sns.stripplot(y=df_Umsaetze["Saldo"],x=df_Umsaetze["Anwender"])


In [ ]:
X = df_Umsaetze.values
model = IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=2).fit(X)

df_model = pd.DataFrame(df_Umsaetze.values, columns = ['Anwender','Umsaetze'])

##Decision_function: as the scores of inliers are close to 0 and the scores of outliers are close to -1.
df_model['scores'] = model.decision_function(X)

## Predict: Returns -1 for outliers and 1 for inliers.
df_model['anomaly'] = model.predict(X)
df_model['Anwender'] = df_model['Anwender'].astype(int)

df_model.head(10)


In [ ]:
ax = sns.stripplot(y=df_model["scores"],x=df_model["Anwender"])